In [15]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Import the datetime module from the datetime library.
from citipy import citipy

from datetime import datetime 
import requests

# Import the API key.
from config import weather_api_key 
# Import linear regression from the SciPy stats module.
from scipy.stats import linregress 

In [72]:
#https://raw.githubusercontent.com/timothykeating/Final-Project/main/pgAdmin_SQLAlchemy/METAREA.csv
metadf = pd.read_csv("https://raw.githubusercontent.com/timothykeating/Final-Project/main/pgAdmin_SQLAlchemy/METAREA.csv") 
metadf

,METAREA,Metropolitan area
0,60,"Abilene, TX"
1,80,"Akron, OH"
2,120,"Albany, GA"
3,160,"Albany-Schenectady-Troy, NY"
4,200,"Albuquerque, NM"
...,...,...
475,9340,"Yuba City, CA"
476,9360,"Yuma, AZ"
477,9997,"Other metropolitan areas, unidentified"
478,9998,"NIU, household not in a metropolitan area"


In [74]:
regex = r'(?P<City>[^,]+)\s*,\s*(?P<State>[^\s]+)' 
#regex = r'(?P<City>[^,]+)\s*,\s*(?P<State>[^\s]+)\s+(?P<Zip>\S+)'

newdf = metadf['Metropolitan area'].str.extract(regex) 
newdf['METAREA'] = metadf['METAREA'] 
newdf

,City,State,METAREA
0,Abilene,TX,60
1,Akron,OH,80
2,Albany,GA,120
3,Albany-Schenectady-Troy,NY,160
4,Albuquerque,NM,200
...,...,...,...
475,Yuba City,CA,9340
476,Yuma,AZ,9360
477,Other metropolitan areas,unidentified,9997
478,NIU,household,9998


In [75]:
newdf1 = newdf.dropna() 
newdf1

,City,State,METAREA
0,Abilene,TX,60
1,Akron,OH,80
2,Albany,GA,120
3,Albany-Schenectady-Troy,NY,160
4,Albuquerque,NM,200
...,...,...,...
474,Youngstown-Warren-Boardman,OH,9321
475,Yuba City,CA,9340
476,Yuma,AZ,9360
477,Other metropolitan areas,unidentified,9997


In [77]:
newdf1.to_csv(r'METAREA_state_city.csv', encoding='utf-8', index=False) 

In [78]:
newdf2 = newdf1.loc[(newdf1['State'] == "NC")] 
newdf2

,City,State,METAREA
16,Asheville,NC,480
69,Burlington,NC,1300
143,Fayetteville,NC,2560
167,Goldsboro,NC,2980
174,Greensboro-Winston Salem,NC,3120
175,Winston-Salem,NC,3121
176,Greensboro-High Point,NC,3122
177,Greenville,NC,3150
193,Hickory-Morganton,NC,3290
194,Hickory-Morganton-Lenoir,NC,3291


In [79]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
   
# declare an empty list to store
# latitude and longitude of values 
# of city column
longitude = []
latitude = []
   
# function to find the coordinate
# of a given city 
def findGeocode(city):
       
    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
    try:
          
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="your_app_name")
          
        return geolocator.geocode(city)
    except GeocoderTimedOut:
          
        return findGeocode(city)    
  
# each value from city column
# will be fetched and sent to
# function find_geocode   
for i in (newdf2["City"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
          
        # coordinates returned from 
        # function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
       
    # if coordinate for a city not
    # found, insert "NaN" indicating 
    # missing value 
    else:
        latitude.append(np.nan)
        longitude.append(np.nan) 

In [80]:
# now add this column to dataframe
newdf2["Longitude"] = longitude
newdf2["Latitude"] = latitude 
newdf2 

<ipython-input-80-2b720f5196c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf2["Longitude"] = longitude
<ipython-input-80-2b720f5196c3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf2["Latitude"] = latitude


,City,State,METAREA,Longitude,Latitude
16,Asheville,NC,480,-82.554016,35.600950
69,Burlington,NC,1300,-73.212906,44.476160
143,Fayetteville,NC,2560,-94.157433,36.062584
167,Goldsboro,NC,2980,-77.992765,35.384884
174,Greensboro-Winston Salem,NC,3120,-80.164593,36.111102
175,Winston-Salem,NC,3121,-80.244052,36.099813
176,Greensboro-High Point,NC,3122,-79.875808,36.025256
177,Greenville,NC,3150,-82.398488,34.851354
193,Hickory-Morganton,NC,3290,-81.771459,35.715361
194,Hickory-Morganton-Lenoir,NC,3291,NaN,NaN


In [81]:
newdf3 = newdf2.dropna() 
newdf3 

,City,State,METAREA,Longitude,Latitude
16,Asheville,NC,480,-82.554016,35.600950
69,Burlington,NC,1300,-73.212906,44.476160
143,Fayetteville,NC,2560,-94.157433,36.062584
167,Goldsboro,NC,2980,-77.992765,35.384884
174,Greensboro-Winston Salem,NC,3120,-80.164593,36.111102
175,Winston-Salem,NC,3121,-80.244052,36.099813
176,Greensboro-High Point,NC,3122,-79.875808,36.025256
177,Greenville,NC,3150,-82.398488,34.851354
193,Hickory-Morganton,NC,3290,-81.771459,35.715361
210,Jacksonville,NC,3600,-81.655651,30.332184


In [82]:
newdf3.to_csv(r'latitude_longitude.csv', encoding='utf-8', index=False) 